In [ ]:
cd /Users/shintaro/Downloads

In [ ]:
#モジュールのインポート
#ModuleNotFoundErrorが出たら当該のモジュールを pip install (モジュール名) でインストールしてください
import csv
import docx2txt
import pandas as pd
import MeCab
from wordcloud import WordCloud
import matplotlib as mpl
import matplotlib.pyplot as plt 
from pathlib import Path 
import mojimoji
import unicodedata
import urllib
import numpy as np
import seaborn as sns

In [ ]:
#ファイル名をひとつひとつ入力する煩雑な作業を省略するため、csvファイルにファイル名を書いておいてそれを読み込みます
comp_fstscreening = pd.read_csv("/Users/shintaro/Downloads/textmining-dx.csv", encoding = 'utf-8')
comp_fstscreening_filename = comp_fstscreening['file_name'].tolist()

In [ ]:
#企業の番号とファイル名が入ったリストcomp_IR_tmpを作成する
comp_IR_tmp = [[]for i in range(len(comp_fstscreening_filename))]
for i in range(len(comp_IR_tmp)):
    comp_IR_tmp[i].append(i)
    comp_IR_tmp[i].append(comp_fstscreening_filename[i])

In [ ]:
#ファイルを読み込む関数read_IRを定義する
def read_IR(comp_num, file_path):
    text = docx2txt.process(file_path)
    text = text.replace('\n','').replace('\u3000','').replace('\xa0','')
    comp_IR[comp_num].append(text)

In [ ]:
comp_IR = [[]for i in range(len(comp_IR_tmp))]

In [ ]:
cd /Users/shintaro/Downloads/textmining-dx

In [ ]:
#それぞれの企業のIRのテキストデータが入ったリストread_IRを作成する
for i in range(len(comp_IR_tmp)):
    read_IR(comp_IR_tmp[i][0], comp_IR_tmp[i][1])

In [ ]:
#文を句点で分割するためのモジュールのインポート
import functools

from ja_sentence_segmenter.common.pipeline import make_pipeline
from ja_sentence_segmenter.concatenate.simple_concatenator import concatenate_matching
from ja_sentence_segmenter.normalize.neologd_normalizer import normalize
from ja_sentence_segmenter.split.simple_splitter import split_newline, split_punctuation

split_punc2 = functools.partial(split_punctuation, punctuations = r"。!?")
concat_tail_te = functools.partial(concatenate_matching, former_matching_rule = r"^(?P<result>.+)(て)$", remove_former_matched = False)
segmenter = make_pipeline(normalize, split_newline, concat_tail_te, split_punc2)

In [ ]:
#分割したものをリストとして保存
lists = [[]for i in range(len(comp_IR))]

for i in range(len(lists)):
    lists[i] = list(segmenter(comp_IR[i]))

In [ ]:
#リストの結合
all_lists = []
for i in range(len(lists)):
    all_lists += lists[i]

In [ ]:
len(all_lists)

In [ ]:
#形態素解析
length = len(all_lists)
    
def get_noun_words_from_sentence(sentence, mecab, stopword_list = []):
    return [
        x.split('\t')[0] for x in mecab.parse(sentence).split('\n') if len(x.split('\t')) > 1 and \
         '名詞' in x.split('\t')[3] and x.split('\t')[0] not in stopword_list
    ]

mecab = MeCab.Tagger('-Ochasen')
nouns = [[] for i in range(length)]
stopword_list = []

for i in range(length):
    
    noun = get_noun_words_from_sentence(sentence = all_lists[i], mecab = mecab, stopword_list = stopword_list)
    nouns[i] += noun

In [ ]:
len_n = len(nouns)

In [ ]:
nouns_com = [[] for i in range(len_n)]

In [ ]:
words_wakati1 = [[]for i in range(len(comp_IR))]

In [ ]:
#記事本文に対して形態素解析を行う関数MeCab_textminingを定義する
def MeCab_textmining(num, text_tmp):
    text = text_tmp
    tagger = MeCab.Tagger("-Owakati")
    str_output = tagger.parse(text)
    words_wakati1[num] = str_output.split(' ') 

In [ ]:
for i in range(len(comp_IR)):
    MeCab_textmining(i, comp_IR[i][0])

In [ ]:
#words_wakati1に入っている要素の数を数えてその値をnとする
n_all = [[]for i in range(len(words_wakati1))]

for i in range(len(words_wakati1)):
    n_all[i] = len(words_wakati1[i])

In [ ]:
words_wakati1

In [ ]:
words_wakati_WordCloud = ''
for i in range(len(words_wakati1)):
    for j in range(len(words_wakati1[i])):
        words_wakati_WordCloud += words_wakati1[i][j]
#words_wakati_WordCloud = words_wakati_WordCloud[0]

In [ ]:
words_wakati_WordCloud

In [ ]:
#下のstop_wordsに出力結果に入れたくない単語を入れると削除される
stop_words = ['は', 'http', 'jp'] #wordcloudの結果に入れたくないwordを決定する
fpath = '/Users/shintaro/Library/Fonts//Arial Unicode.ttf'  # 日本語フォント指定(フォントのファイルが入っている場所を探し、そのフルパスを入れてください)
 
wordcloud = WordCloud(
    font_path = fpath,
    width = 900, height = 600,   # default width=400, height=200
    background_color = "white",   # default=”black”
    stopwords = set(stop_words),
    max_words = 500,   # default=200
    min_font_size = 4,   #default=4
    collocations = False   #default = Ture
    ).generate(words_wakati_WordCloud)

wordcloud.to_file("./test.png") #wordcloudの結果を./以下の部分を名前としてpng形式で保存する

In [ ]:
#企業名のリストからstopwordsに該当するものを除外する
words_wakati_WordCloud1 = words_wakati_WordCloud.split()
target = stop_words
L = len(stop_words)
for i in range(L):
    
    while target[i] in words_wakati_WordCloud1:
        words_wakati_WordCloud1.remove(target[i])

In [ ]:
#同一名称の成分を除外する
wwW_unique = list(set(words_wakati_WordCloud1))
#wwW_unique

In [ ]:
#words_wakati_WordCloud2を作成
words_wakati_WordCloud2 = words_wakati_WordCloud.split()
target = stop_words
L = len(stop_words)
for i in range(L):
    
    while target[i] in words_wakati_WordCloud2:
        words_wakati_WordCloud2.remove(target[i])
        
import collections 
c = collections.Counter(words_wakati_WordCloud2)
c = c.most_common()

In [ ]:
c[0]

In [ ]:
#単語２つの組み合わせを作成する
import itertools
combination_sentences = [[] for i in range(length)]

combination_sentences = [list(itertools.combinations(words, 2)) for words in nouns_com]

In [ ]:
combination_sentences = [[tuple(sorted(combi)) for combi in combinations] for combinations in combination_sentences]

In [ ]:
tmp = []
for combinations in combination_sentences:
    tmp.extend(combinations)

In [ ]:
combination_sentences = tmp

In [ ]:
#jaccard係数を求める
def make_jaccard_coef_data(combination_sentences):

    combi_count = collections.Counter(combination_sentences)

    word_associates = []
    for key, value in combi_count.items():
        word_associates.append([key[0], key[1], value])

    word_associates = pd.DataFrame(word_associates, columns = ['word1', 'word2', 'intersection_count'])

    words = []
    for combi in combination_sentences:
        words.extend(combi)

    word_count = collections.Counter(words)
    word_count = [[key, value] for key, value in word_count.items()]
    word_count = pd.DataFrame(word_count, columns = ['word', 'count'])

    word_associates = pd.merge(
        word_associates,
        word_count.rename(columns = {'word': 'word1'}),
        on = 'word1', how = 'left'
    ).rename(columns = {'count': 'count1'}).merge(
        word_count.rename(columns = {'word': 'word2'}),
        on = 'word2', how = 'left'
    ).rename(columns = {'count': 'count2'}).assign(
        union_count = lambda x: x.count1 + x.count2 - x.intersection_count
    ).assign(jaccard_coef = lambda x: x.intersection_count / x.union_count).sort_values(
        ['jaccard_coef', 'intersection_count'], ascending = [False, False]
    )
    
    return word_associates

jaccard_coef_data = make_jaccard_coef_data(combination_sentences)

In [ ]:
#空のデータフレームjaccard_coef_data1の作成
col = ['word1',	'word2', 'intersection_count', 'count1', 'count2', 'union_count', 'jaccard_coef']
jaccard_coef_data1 = pd.DataFrame(index = [], columns = col)

In [ ]:
#二つの単語が同一の際にその行を削除する
for i in range(len(jaccard_coef_data)):
    if jaccard_coef_data.iat[i, 0] != jaccard_coef_data.iat[i, 1]:
        jaccard_coef_data1 = jaccard_coef_data1.append(jaccard_coef_data.iloc[i, :])

In [ ]:
#ジャッカード係数の下限を設定
jaccard_coef_min = 0.05

In [ ]:
#空のデータフレームjaccard_coef_data2の作成
col = ['word1',	'word2', 'intersection_count', 'count1', 'count2', 'union_count', 'jaccard_coef']
jaccard_coef_data2 = pd.DataFrame(index = [], columns = col)

In [ ]:
#ジャッカード係数の値がjaccard_coef_min以下の行を削除する
for i in range(len(jaccard_coef_data1)):
    if jaccard_coef_data1.iat[i, 6] >= jaccard_coef_min:
        jaccard_coef_data2 = jaccard_coef_data2.append(jaccard_coef_data1.iloc[i, :])

In [ ]:
#jaccard_coef_data.to_csv("jaccard_coef.csv")

In [ ]:
jaccard_coef_data2